# Anonymise data

This notebook was used to anonymise the data collected from both the surveys and conversation logs 
The original files have been remo

In [1]:
import json
import pickle
from pandas.io.json import json_normalize
import os
import logging
import logging.handlers
import pandas as pd
import numpy as np
import glob
from datetime import datetime,timedelta
from collections import Counter
import nltk 
from functools import reduce
import re
from scipy.stats import pointbiserialr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
matplotlib.style.use('ggplot')
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

### Read in dialog files (JSON)



Each user utterance in a conversation is stored in a separate JSON file which contains fields about the message, the user, and the conversation:

Message:
- type
- ID
- timestamp
- Text

User:
- ID
- Name

Conversation:
- ID

Bot:
- ID
- Name



In [2]:
def dt_parse(t):
    ''' 
    Function that parses datetime so it can be used to find duration of conversation
    INPUT: a string t representing a message timestamp
    OUTPUT: a string ret representing the message timestamp in the format %Y-%m-%dT%H:%M
    '''
    ret = datetime.strptime(t[0:16],'%Y-%m-%dT%H:%M')
    return ret 

In [3]:
def create_utterance_df(rootDir):
    '''
    Function that traverses subfolders of a root directory 
    where each subfolder corresponds to a conversation with one of the chatbots.
    Each subfolder contains JSON files, each corresponding to a single user message. 
    '''
    
    jsdata = []
    df = pd.DataFrame(pd.np.empty((0, 7)))
    df.columns = ['Username','Duration', 'Chatbot', 'NumberOfTurns', 'Utterance','Word_Count', "Avg_Utt_Len"]

    for subdir, dirs, files in os.walk(rootDir): # each conversation has its own folder of json files
        for dir_ in dirs: # for each conversation
            message_timestamps = []
            user_utterances = []
            char_counts = []
            numberOfTurns = 0
            word_count = 0
            for file in glob.glob(rootDir+dir_+'/'+'*.json'):
                with open(file) as f:
                    data = json.load(f)
                    numberOfTurns += 1
                    message_timestamps.append(dt_parse(data['timestamp']))
                    user_utterances.append(data['text'])
                    for word in (data['text']).split():
                        word_count += 1
                    char_counts.append(len(data['text']) - data['text'].count(' '))
            avg_char_count = np.mean(char_counts)       


            #add post interaction survey (Nasoto)

            #Duration
            maxTime = max(dt for dt in message_timestamps)
            minTime = min(dt for dt in message_timestamps)
            duration =  maxTime - minTime

            #get the information from nasoto inserted
            jsdata.append({'Username': data['from']['name'],
                           'Duration': duration,'Chatbot': data['recipient']['name'], 
                           'NumberOfTurns' : numberOfTurns, 'Utterance': user_utterances, 
                            'Word_Count': word_count, 'Avg_Utt_Len': avg_char_count})



    df =  pd.DataFrame.from_records(jsdata)
    df[:]['Duration'] / np.timedelta64(1, 'm')
    return df  

## Collect Pre-Interaction Data

In [4]:
nasoto_utterances = '' 
makoto_utterances = ''
df_nasoto = create_utterance_df(nasoto_utterances)
df_makoto = create_utterance_df(makoto_utterances)

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



KeyError: 'Duration'

# Collect Post-Interaction Data \& Anonymise

In [ ]:
PostMakotoDir = ''
df_PostMakoto = pd.read_csv(PostMakotoDir, error_bad_lines=False)
df_PostMakoto['Username'] =  df_PostMakoto['Username'].str.replace(r'@ucdconnect.ie', '')

In [ ]:
PostNasotoDir = ''
df_PostNasoto = pd.read_csv(PostNasotoDir, error_bad_lines=False)
df_PostNasoto['Username'] =  df_PostNasoto['Username'].str.replace(r'@ucdconnect.ie', '')

## Collect Demographic Data \& Anonymise

In [ ]:
PreInteractionPath = ''
df_dem = pd.read_csv(PreInteractionPath,  error_bad_lines=False)
df_dem['Username'] =  df_dem['Username'].str.replace(r'@ucdconnect.ie', '')

In [ ]:
Survey4Dir = ''
df_survey4 = pd.read_csv(Survey4Dir)
df_survey4['Username'] =  df_survey4['Username'].str.replace(r'@ucdconnect.ie', '')

# Create Dataframes

In [ ]:
df_allSurveys =  [df_dem, df_PostMakoto, df_PostNasoto, df_survey4, df_nasoto, df_makoto]


df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Username'],
                                            how='outer'), df_allSurveys)

In [ ]:
df_merged['Chatbot_binary'] = pd.np.where(df_merged['Which chatbot did you prefer interacting with?'].str.contains("Makoto"), 0, 1)

# Anonymise username \& Convert to Pickle File 

In [ ]:
df_merged.drop(columns=["Username"], inplace=True)
# df_merged.to_pickle("./df_merged.pkl")

In [ ]:
df_dem.drop(columns=["Username"], inplace= True)
# df_dem.to_pickle("./df_dem.pkl")

In [ ]:
frames = [df_nasoto, df_makoto]
df_chatbotdata =  pd.concat(frames)
df_chatbotdata.drop(columns=["Username"], inplace= True)
# df_chatbotdata.to_pickle("./df_chatbotdata.pkl")